In [40]:
import os
import duckdb
import pandas as pd
import pyarrow.parquet as pq
from dotenv import load_dotenv

In [41]:
load_dotenv()

True

In [42]:
conn = duckdb.connect(database=':memory:')

#### KM rodado

In [69]:
km_df = pd.DataFrame()

In [70]:
folder_path = '/home/marcelo-borges/Documentos/Projetos/datalakehouse_lowcost/elt/tmp'

In [71]:
km_df=conn.execute(f""
                f"SELECT * FROM read_parquet('{folder_path}/dna_kmRodado_*.parquet')"
                f"order by DATA, PREFIXO_CARRO").fetchdf()

In [72]:
km_df.sample(10)

,PREFIXO_CARRO,ID_EQUIPAMENTO,DATA,ID_LINHA,DS_TIPO_CHASSI,DS_TIPO_MOTOR,DS_TIPO_CAMBIO,CARRO_AUXILIAR,FX_HORARIA,FX_SIGLA,FX_NOME,KM_OCIOSA,LITROS_LINHA,KM_LINHA,ABS_CARRO_KM_TOTAL_DIA,ABS_CARRO_LITROS_TOTAL_DIA
597782,20446,8793,29/06/2023,5513,OF-1721,OM-924-LA.METÁLICO,MANUAL,NAO,5,PM,2 PICO DA MANHA,"1,17684893675780171223418945042805854736","6,47020712510356255178127589063794531898","19,00684893675780171223418945042805854737",193,"65,7"
459030,20498,8876,22/10/2023,5297,OF-1721,OM-924-LA.METÁLICO,MANUAL,NAO,7,PM,2 PICO DA MANHA,"8,65866156787762906309751434034416826004","14,39724665391969407265774378585086042065","48,49866156787762906309751434034416826004",191,"56,7"
233497,20360,1429,12/02/2023,5350,17-230 EOD,MWM 6.12 TCAE-EURO III,MANUAL,NAO,6,PM,2 PICO DA MANHA,",8422227781599849896804052786290574770153","18,3122521733691913190318343861404715742","44,14222277815998498968040527862905747701",163,"67,62"
291260,20086,8178,14/11/2023,5595,OF-1721,OM-924-LA.METÁLICO,MANUAL,NAO,12,EP,3 ENTRE PICO,"6,5190766805539916676049994370003377998","16,98979844612093232744060353563787861728","40,19907668055399166760499943700033779981",212,"89,6"
208426,20444,8790,10/11/2023,5440,OF-1721,OM-924-LA.METÁLICO,MANUAL,NAO,6,PM,2 PICO DA MANHA,"8,79845226038118037013166375103581622318","16,40576604364239020348034250989779946597","45,70845226038118037013166375103581622317",269,"96,55"
229485,20028,8101,11/12/2023,5554,OF-1721,OM-924-LA.METÁLICO,MANUAL,NAO,5,PM,2 PICO DA MANHA,"8,89041606886657101865136298421807747489","20,03221807747489239598278335724533715925","53,47041606886657101865136298421807747488",209,"78,3"
272995,20344,1386,13/12/2023,5392,17-230 EOD,MWM 6.12 TCAE-EURO III,MANUAL,NAO,21,NT,5 NOTURNO,",8652300737312308271890640977342446585223","9,02534847424788762714600936440449921964","23,22523007373123082718906409773424465853",193,75
558043,20495,8873,27/07/2023,5263,OF-1721,OM-924-LA.METÁLICO,MANUAL,NAO,11,EP,3 ENTRE PICO,"3,5403149521659679673223691282382027303","14,5768784263140922283134472750725572396","38,69031495216596796732236912823820273029",256,"96,45"
364775,20345,1388,18/04/2023,5097,17-230 EOD,MWM 6.12 TCAE-EURO III,MANUAL,NAO,7,PM,2 PICO DA MANHA,"4,85948488558244655805631415387512948489","17,39702891044354458988605330068744702891","43,62948488558244655805631415387512948489",239,"95,3"
609921,20029,8076,30/01/2023,5112,OF-1721,OM-924-LA.METÁLICO,MANUAL,NAO,19,NT,5 NOTURNO,"4,97651529193697868396663577386468952734","18,28035217794253938832252085264133456904","50,93651529193697868396663577386468952733",287,103


In [73]:
km_df.columns

Index(['PREFIXO_CARRO', 'ID_EQUIPAMENTO', 'DATA', 'ID_LINHA', 'DS_TIPO_CHASSI',
       'DS_TIPO_MOTOR', 'DS_TIPO_CAMBIO', 'CARRO_AUXILIAR', 'FX_HORARIA',
       'FX_SIGLA', 'FX_NOME', 'KM_OCIOSA', 'LITROS_LINHA', 'KM_LINHA',
       'ABS_CARRO_KM_TOTAL_DIA', 'ABS_CARRO_LITROS_TOTAL_DIA'],
      dtype='object')

Entender o cenário onde pode ter 1 id_equipamento para mais de 1 prefixo_carro ou vice-versa. Porque em 1 ano isso não ocorreu e mesmo assim, quando forem analisar resultados sempre olharão em nível de prefixo de carro, minha sugestão é tirar o id_equipamento.

In [74]:
valores_unicos = km_df[['PREFIXO_CARRO', 'ID_EQUIPAMENTO']].drop_duplicates()

In [75]:
km_df.drop(columns=['ID_EQUIPAMENTO'], inplace=True)

In [76]:
carros = km_df[['PREFIXO_CARRO','DS_TIPO_CHASSI', 'DS_TIPO_CAMBIO','DS_TIPO_MOTOR','CARRO_AUXILIAR']].drop_duplicates()

In [118]:
carros

PREFIXO_CARRO     object
DS_TIPO_CHASSI    object
DS_TIPO_CAMBIO    object
DS_TIPO_MOTOR     object
CARRO_AUXILIAR    object
dtype: object

In [78]:
carros.to_parquet('/home/marcelo-borges/Documentos/Projetos/datalakehouse_lowcost/elt/transmission/dimension/dim_carro.parquet', index=False)

OSError: Cannot save file into a non-existent directory: '/home/marcelo-borges/Documentos/Projetos/datalakehouse_lowcost/elt/transmission/dimension'

In [79]:
km_df.drop(columns=['DS_TIPO_CHASSI', 'DS_TIPO_CAMBIO','CARRO_AUXILIAR','DS_TIPO_MOTOR'], inplace=True)

exclusão das colunas ABS_CARRO_KM_TOTAL E ABS_CARRO_LITROS, pois são valores repetidos obtidos pelo agrupamento das colunas LITROS_LINHA E KM_LINHA

In [80]:
km_df.drop(columns=['ABS_CARRO_KM_TOTAL_DIA', 'ABS_CARRO_LITROS_TOTAL_DIA'], inplace=True)

criação da tabela de horário e exclusão de informações redundantes

In [81]:
horarios = km_df[['FX_HORARIA','FX_SIGLA','FX_NOME']].drop_duplicates()

In [82]:
horarios.to_parquet('/home/marcelo-borges/Documentos/Projetos/datalakehouse_lowcost/elt/transmission/dimension/dim_horario.parquet', index=False)

OSError: Cannot save file into a non-existent directory: '/home/marcelo-borges/Documentos/Projetos/datalakehouse_lowcost/elt/transmission/dimension'

In [83]:
km_df.drop(columns=['FX_SIGLA','FX_NOME'], inplace=True)

In [84]:
horarios    

,FX_HORARIA,FX_SIGLA,FX_NOME
0,18,PT,4 PICO DA TARDE
1,15,EP,3 ENTRE PICO
2,14,EP,3 ENTRE PICO
3,11,EP,3 ENTRE PICO
4,6,PM,2 PICO DA MANHA
5,10,EP,3 ENTRE PICO
6,20,NT,5 NOTURNO
7,13,EP,3 ENTRE PICO
8,5,PM,2 PICO DA MANHA
9,9,EP,3 ENTRE PICO


casting

In [85]:
km_df

,PREFIXO_CARRO,DATA,ID_LINHA,FX_HORARIA,KM_OCIOSA,LITROS_LINHA,KM_LINHA
0,20017,01/01/2023,5274,18,"1,39773851590106007067137809187279151943","8,40282685512367491166077738515901060071","28,07773851590106007067137809187279151944"
1,20017,01/01/2023,5271,15,",8010278076509448455984022123214011368877","4,81556306652327546474112766938085727454","16,09102780765094484559840221232140113689"
2,20017,01/01/2023,5269,14,",8036472576432631740666769088953756337384","4,83131049316331233676447995083730219696","16,14364725764326317406667690889537563375"
3,20017,01/01/2023,5269,11,",8036472576432631740666769088953756337384","4,83131049316331233676447995083730219696","16,14364725764326317406667690889537563375"
4,20017,01/01/2023,5147,6,"1,41188354585957904440006145337225380243","8,48786295897987402058687970502381318175","28,36188354585957904440006145337225380244"
...,...,...,...,...,...,...,...
644537,20552,31/12/2023,5261,12,"5,48318488996707676312597470109166522267","10,61323167561947669381389707156472015249","33,02318488996707676312597470109166522268"
644538,20552,31/12/2023,5265,4,"5,5130497314156991855830878530583954254","10,67103794836250216600242592271703344308","33,20304973141569918558308785305839542541"
644539,20552,31/12/2023,5263,18,"6,99832784612718766245018194420377750823","13,54593657944896898284526078669208109513","42,14832784612718766245018194420377750821"
644540,20552,31/12/2023,5264,10,"7,32485011263212614798128573904002772483","14,17795182810604747877317622595737307226","44,11485011263212614798128573904002772483"


In [86]:
km_df.dtypes

PREFIXO_CARRO    object
DATA             object
ID_LINHA         object
FX_HORARIA       object
KM_OCIOSA        object
LITROS_LINHA     object
KM_LINHA         object
dtype: object

In [92]:
# km_df['DATA'] = pd.to_datetime(km_df['DATA'], format='%d/%m/%Y')
km_df['DATA'] = pd.to_datetime(km_df['DATA'], format='%Y-%m-%d %H:%M:%S')

# km_df['DATA'] = km_df['DATA'].dt.strftime('%d/%m/%Y')

In [93]:
km_df

,PREFIXO_CARRO,DATA,ID_LINHA,FX_HORARIA,KM_OCIOSA,LITROS_LINHA,KM_LINHA
0,20017,2023-01-01,5274,18,"1,39773851590106007067137809187279151943","8,40282685512367491166077738515901060071","28,07773851590106007067137809187279151944"
1,20017,2023-01-01,5271,15,",8010278076509448455984022123214011368877","4,81556306652327546474112766938085727454","16,09102780765094484559840221232140113689"
2,20017,2023-01-01,5269,14,",8036472576432631740666769088953756337384","4,83131049316331233676447995083730219696","16,14364725764326317406667690889537563375"
3,20017,2023-01-01,5269,11,",8036472576432631740666769088953756337384","4,83131049316331233676447995083730219696","16,14364725764326317406667690889537563375"
4,20017,2023-01-01,5147,6,"1,41188354585957904440006145337225380243","8,48786295897987402058687970502381318175","28,36188354585957904440006145337225380244"
...,...,...,...,...,...,...,...
644537,20552,2023-12-31,5261,12,"5,48318488996707676312597470109166522267","10,61323167561947669381389707156472015249","33,02318488996707676312597470109166522268"
644538,20552,2023-12-31,5265,4,"5,5130497314156991855830878530583954254","10,67103794836250216600242592271703344308","33,20304973141569918558308785305839542541"
644539,20552,2023-12-31,5263,18,"6,99832784612718766245018194420377750823","13,54593657944896898284526078669208109513","42,14832784612718766245018194420377750821"
644540,20552,2023-12-31,5264,10,"7,32485011263212614798128573904002772483","14,17795182810604747877317622595737307226","44,11485011263212614798128573904002772483"


In [ ]:
km_df['PREFIXO_CARRO'] = pd.to_numeric(km_df['PREFIXO_CARRO'],downcast='integer')

In [126]:
km_df.dtypes

PREFIXO_CARRO            object
DATA             datetime64[ns]
ID_LINHA                 object
FX_HORARIA               object
KM_OCIOSA                object
LITROS_LINHA             object
KM_LINHA                 object
dtype: object

In [129]:
km_df['FX_HORARIA'] = pd.to_numeric(km_df['FX_HORARIA'],downcast='integer')

ValueError: Unable to parse string "REAB" at position 114

Transformando os dados de litros_linha que está em string para o formato númerico universal

In [63]:
km_df['LITROS_LINHA'] = km_df['LITROS_LINHA'].str.replace(',', '.')
km_df['LITROS_LINHA'] = pd.to_numeric(km_df['LITROS_LINHA'], downcast='float')

In [64]:
km_df['KM_OCIOSA'] = km_df['KM_OCIOSA'].str.replace(',', '.')
km_df['KM_OCIOSA'] = pd.to_numeric(km_df['KM_OCIOSA'], downcast='float')

In [65]:
km_df['KM_LINHA'] = km_df['KM_LINHA'].str.replace(',', '.')
km_df['KM_LINHA'] = pd.to_numeric(km_df['KM_LINHA'], downcast='float')

In [67]:
km_df.sample(10)

,PREFIXO_CARRO,DATA,ID_LINHA,FX_HORARIA,KM_OCIOSA,LITROS_LINHA,KM_LINHA
277375,20040,2023-03-14,5069,16,5.304724,6.664969,19.309723
138502,20527,2023-07-07,5077,6,2.025820,14.195736,32.045822
404026,20451,2023-02-20,5356,23,4.957964,9.353942,30.397964
16260,20386,2023-11-01,5281,16,4.729482,13.592973,29.929482
559102,20503,2023-08-27,5440,19,3.943256,12.799377,40.853256
137656,20356,2023-07-07,5451,7,0.062833,5.733990,15.312833
183961,20435,2023-10-09,5455,5,2.546889,7.149055,17.096889
249941,20354,2023-12-12,5556,19,4.148195,11.635033,32.088196
590542,20519,2023-12-28,5263,18,10.798631,16.805677,45.948631
156903,20472,2023-06-08,5505,20,0.980557,10.361510,25.400557


Conclusão da tabela fato km_rodado

In [58]:
km_df.to_parquet('/home/marcelo-borges/Documentos/Projetos/datalakehouse_lowcost/elt/transmission/fact/fact_km_rodado.parquet')

#### Tabela dimensão de Linha

In [96]:
linha_df=conn.execute(f""
                f"SELECT * FROM read_parquet('{folder_path}/dna_linha_*.parquet')"
                f"order by DATA_LINHA").fetchdf()

exclusão de todas as colunas que não são pertinentes a dimensão linha

In [102]:
linha_df.drop(columns=['ID_LINHA_BI','DATA_LINHA','DIA','MES','ANO','ID_EMPRESA','CODIGO_TN','NOME_TN'], inplace=True)

In [103]:
linha_df.drop_duplicates(inplace=True)

In [105]:
linha_df.sample(5)

,ID_LINHA,LINHA_NUM,LINHA_ALFA,DESCRICAO_LINHA,TIPO_LINHA,LOCAL_DE_CONTROLE,CODIGO_UN,NOME_UN,CODIGO_UGB,NOME_UGB
300,5369,213,E,T. BIBLIA / T. ARAGUAIA / HIDROLANDIA,ALIMENTADORA,T. PRAÇA DA BÍBLIA,005,SUDESTE,0213,T. BIBLIA / HIDROLÂNDIA
597,5452,26,C,PRACA DELMIRO / T. BANDEIRAS,TRONCAL,T. BANDEIRAS,001,SUDOESTE,0026,TERMINAL BANDEIRAS / FLAMBOYANT - VIA AV.13
78,5029,401,C,T. BIBLIA / T. PRACA A (VIA FEIRA HIPPIE),CIRCULAR,T. PRACA A,001,SUDOESTE,0401,CIRCULAR - AV. INDEPENDENCIA - SENTIDO ANTI-HO...
316,5385,9903,,COPA DA ESTRATEGIA - TURMA III,ESPECIAL,GARAGEM HP (CENTRAL),001,SUDOESTE,9900,COPA DA ESTRATEGIA 2022
581,5422,570,B,T. MARANATA / MADRE GERMANA I - VIA SAO CONRADO,ALIMENTADORA,T. MARANATA,001,SUDOESTE,0570,MADRE GERMANA I - TERMINAL MARANATA


In [108]:
linha_df.dtypes

ID_LINHA              int16
LINHA_NUM             int16
LINHA_ALFA           object
DESCRICAO_LINHA      object
TIPO_LINHA           object
LOCAL_DE_CONTROLE    object
CODIGO_UN            object
NOME_UN              object
CODIGO_UGB           object
NOME_UGB             object
dtype: object

In [107]:
linha_df['ID_LINHA'] = pd.to_numeric(linha_df['ID_LINHA'], downcast='integer')
linha_df['LINHA_NUM'] = pd.to_numeric(linha_df['LINHA_NUM'], downcast='integer')

In [63]:
linha_df.to_parquet('/home/marcelo-borges/Documentos/Projetos/datalakehouse_lowcost/elt/transmission/dimension/dim_linha.parquet')

#### Tabela dimensão de Calendário

In [110]:
folder_path = '/home/marcelo-borges/Documentos/Projetos/datalakehouse_lowcost/elt/tmp/'

In [111]:
calendar_df=conn.execute(f""
                f"SELECT * FROM read_parquet('{folder_path}dna_calendario_*.parquet')"
                f"order by ID_CALENDARIO ").fetchdf()

In [112]:
calendar_df.drop(columns=['ID_EMPRESA', 'NOME_EMPRESA','SIGLA_EMPRESA'], inplace=True)

casting

In [113]:
calendar_df['DATA'] = pd.to_datetime(calendar_df['DATA'], format='%d/%m/%Y')
calendar_df['DATA'] = pd.to_datetime(calendar_df['DATA'], format='%Y-%m-%d %H:%M:%S')
# calendar_df['DATA'] = calendar_df['DATA'].dt.strftime('%d/%m/%Y')

In [114]:
calendar_df['DIA'] = calendar_df['DATA'].dt.day
calendar_df['MES'] = calendar_df['DATA'].dt.month
calendar_df['ANO'] = calendar_df['DATA'].dt.year

In [115]:
calendar_df.dtypes

DATA                  datetime64[ns]
ID_CALENDARIO                 object
ID_OPERACAO_TIPO              object
NOME_OPERACAO_TIPO            object
DIA                            int32
MES                            int32
ANO                            int32
dtype: object

casting

In [116]:
calendar_df['ID_CALENDARIO'] = pd.to_numeric(calendar_df['ID_CALENDARIO'], downcast='integer')
calendar_df['ID_OPERACAO_TIPO'] = pd.to_numeric(calendar_df['ID_OPERACAO_TIPO'], downcast='integer')

In [108]:
calendar_df.to_parquet('/home/marcelo-borges/Documentos/Projetos/datalakehouse_lowcost/elt/transmission/dimension/dim_calendar.parquet')

In [137]:
import duckdb
import os
from dotenv import load_dotenv
load_dotenv()

duckdb.sql('INSTALL httpfs')
duckdb.sql('LOAD httpfs')
hmac_access = os.getenv('GOOGLE_ACCESS_KEY')
hmac_secret = os.getenv('GOOGLE_SECRET')
duckdb.sql(f"SET s3_access_key_id='{hmac_access}';")
duckdb.sql(f"SET s3_secret_access_key='{hmac_secret}';")
duckdb.sql("SET s3_endpoint='storage.googleapis.com'")
df = duckdb.sql("SELECT * FROM 's3://mvp-hp-transformed/km/*/*/*.parquet'").to_df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [138]:
df

,PREFIXO_CARRO,DATA,ID_LINHA,FX_HORARIA,KM_OCIOSA,LITROS_LINHA,KM_LINHA
0,20289,2022-12-31,5066,7,11.144882,19.158485,61.114883
1,20317,2022-12-31,5070,13,3.970000,13.138386,39.700001
2,20118,2022-12-31,5075,8,3.971599,15.424965,39.221600
3,20511,2022-12-31,5264,8,3.677321,13.802547,40.467319
4,20511,2022-12-31,5264,11,3.677321,13.802547,40.467319
...,...,...,...,...,...,...,...
606713,20052,2024-01-09,5634,15,12.381925,12.531186,23.611925
606714,20077,2024-01-09,5428,14,1.563874,4.518490,11.863874
606715,20077,2024-01-09,5605,13,4.084293,11.800716,30.984293
606716,20085,2024-01-09,5453,11,3.498575,4.045485,12.153575


In [165]:
from google.cloud import bigquery
from dotenv import load_dotenv
load_dotenv()
# Carregar variáveis de ambiente
project_id = os.getenv("GOOGLE_PROJECT_ID")

# Nome completo da tabela
fully_qualified_table_id = "mvp-hp.dados_hp.km"  # Use o ID da tabela diretamente

# Criar um cliente BigQuery
client = bigquery.Client(project=project_id)


In [166]:
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.PARQUET,
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND,
    autodetect=True  # Detectar esquema automaticamente
)


In [168]:
uri = "gs://mvp-hp-transformed/km/*/*/*.parquet"

load_job = client.load_table_from_uri(
    uri,
    fully_qualified_table_id,  # Use o nome completo da tabela diretamente
    job_config=job_config
)

# Aguardar a conclusão do job
load_job.result()

print(f"Os dados foram carregados na tabela {fully_qualified_table_id}.")

NotFound: 404 Not found: Uris gs://mvp-hp-transformed/km/*/*/*.parquet

In [172]:
import os
import pandas as pd
from google.cloud import bigquery

# Carregar variáveis de ambiente
project_id = os.getenv("GOOGLE_PROJECT_ID")
dataset_id = "dados_hp"
table_id = "km"

# Criar um cliente BigQuery
client = bigquery.Client(project=project_id)

# Consulta para atualizar a tabela
query = f"""
INSERT INTO `mvp-hp.dados_hp.km`
SELECT
  *
FROM
  `gs://bucket_name/pattern`
WHERE
  DATA > '2023-08-08'
"""
# Executar a consulta
job = client.query(query)
# Aguardar a conclusão do job
job.result()
# Imprimir um resumo do job
print(job.result())


BadRequest: 400 Table "gs://bucket_name/pattern" must be qualified with a dataset (e.g. dataset.table).

Location: us-east1
Job ID: f6622f46-6284-400a-82b2-97e3bdfafd3d


In [173]:

from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# TODO(developer): Set dataset_id to the ID of the dataset to fetch.
# dataset_id = 'your-project.your_dataset'

dataset = client.get_dataset(dataset_id)  # Make an API request.

full_dataset_id = "{}.{}".format(dataset.project, dataset.dataset_id)
friendly_name = dataset.friendly_name
print(
    "Got dataset '{}' with friendly_name '{}'.".format(
        full_dataset_id, friendly_name
    )
)

# View dataset properties.
print("Description: {}".format(dataset.description))
print("Labels:")
labels = dataset.labels
if labels:
    for label, value in labels.items():
        print("\t{}: {}".format(label, value))
else:
    print("\tDataset has no labels defined.")

# View tables in dataset.
print("Tables:")
tables = list(client.list_tables(dataset))  # Make an API request(s).
if tables:
    for table in tables:
        print("\t{}".format(table.table_id))
else:
    print("\tThis dataset does not contain any tables.")

Got dataset 'mvp-hp.dados_hp' with friendly_name 'None'.
Description: None
Labels:
	Dataset has no labels defined.
Tables:
	calendario
	km
	tmp_carros
	tmp_horarios
	tmp_linha
